In [71]:
import numpy as np
import pandas as pd
import pickle

In [72]:
%pwd

'/home/jupyter-ajakusov/pump/Colab_Analysis'

In [73]:
result_dir = '../encoders'

In [74]:
results = {}
results['rs_01'] = pickle.load(open(result_dir + "/encoder_rs1.pickle",'rb'))


In [144]:
results['rs_02'] = pickle.load(open(result_dir + "/encoder_rs2.pickle",'rb'))
results['rs_03'] = pickle.load(open(result_dir + "/encoder_rs3.pickle",'rb'))


UnpicklingError: invalid load key, '\xd9'.

In [75]:
seed = 'rs_01' 
print('Selected seed:',seed)

Selected seed: rs_01


In [76]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


In [77]:
#Extracted from DeepType/model.py
import os
import argparse
import numpy as np
#import pandas as pd



######################################################################################################################

class AutoEncoder(object):

  _weights_str = "w_{0}"
  _biases_str = "b_{0}"

  def __init__(self, shape):
      '''Autoencoder initializer

      Args:
      shape: list of ints specifying
            num input, hidden1 units,...hidden_n units, num logits
      sess: tensorflow session object to use
      '''
      self.__shape = shape  # [input_dim,hidden1_dim,...,hidden_n_dim,output_dim]
      self.__num_hidden_layers = len(self.__shape) - 2

      self.__variables = {}


      self._setup_variables()


  @property
  def shape(self):
      return self.__shape

  @property
  def num_hidden_layers(self):
      return self.__num_hidden_layers



  def __getitem__(self, item):
      """Get autoencoder tf variable

      Returns the specified variable created by this object.
      Names are weights#, biases#, biases#_out, weights#_fixed,
      biases#_fixed.

      Args:
      item: string, variables internal name
      Returns:
      Tensorflow variable
      """
      return self.__variables[item]

  def __setitem__(self, key, value):
      """Store a tensorflow variable

      NOTE: Don't call this explicity. It should
      be used only internally when setting up
      variables.

      Args:
      key: string, name of variable
      value: tensorflow variable
      """
      self.__variables[key] = value

  def _copy_from_other(self, ae_other):
      for i in range(self.__num_hidden_layers + 1):
          # Train weights
          name_w = self._weights_str.format(i + 1)
          self[name_w] = ae_other[name_w]
          # Train biases
          name_b = self._biases_str.format(i + 1)
          self[name_b] = ae_other[name_b]


  def _copy_from_initialize(self, ae_layer_list):

      for i in range(self.__num_hidden_layers + 1):
          # Train weights
          name_w = self._weights_str.format(i + 1)
          self[name_w] = ae_layer_list[name_w]
          # Train biases
          name_b = self._biases_str.format(i + 1)
          tmp = np.reshape(ae_layer_list[name_b], (1, -1))
          print(tmp[0].shape)
          self[name_b] = tmp[0]




  def get_variables_to_init(self, n):
      """Return variables that need initialization

      Args:
      n: initialize to the n-th layer
      """
      assert n > 0
      assert n <= self.__num_hidden_layers + 1

      vars_to_init = [self._w(n), self._b(n)]

      return vars_to_init


  def _setup_variables(self, initialize = False, initialize_data = []):
      for i in range(0, self.__num_hidden_layers + 1):
          # Trainable weights
          name_w = self._weights_str.format(i + 1)
          w_shape = (self.__shape[i], self.__shape[i + 1])
          if initialize:
              w_init = tf.Variable(initialize_data[name_w])
          else:
              w_init = tf.Variable(tf.random_normal(w_shape, stddev=0.02))


          self[name_w] = tf.Variable(w_init, name=name_w, trainable=True)

          # Trainable biases
          name_b = self._biases_str.format(i + 1)
          b_shape = (self.__shape[i + 1],)
          if initialize:
              tmp = np.reshape(initialize_data[name_b], (1, -1))
              b_init = tf.Variable(tmp[0])
          else:
              b_init = tf.Variable(tf.random_normal(b_shape,  stddev=0.02))
          self[name_b] = tf.Variable(b_init, name=name_b, trainable=True)


  def _w(self, n, suffix=""):
      return self[self._weights_str.format(n) + suffix]

  def _b(self, n, suffix=""):
      return self[self._biases_str.format(n) + suffix]

  @staticmethod
  def _activate(x, w, b,  is_target = False):
      if is_target == False:
          y = tf.nn.sigmoid(tf.add(tf.matmul(x, w), b)) # sigmoid, can change to ReLU
          y = tf.maximum(y, 1.e-9)
          y = tf.minimum(y, 1 - 1.e-9)

      else:
          y = tf.nn.softmax(tf.add(tf.matmul(x, w), b))
      return y


  def supervised_net(self, input_pl, n):
      """Get the supervised fine tuning net

      Args:
      input_pl: tf placeholder for ae input data
      Returns:
      Tensor giving full ae net
      """

      last_output = input_pl
      assert n > 0
      assert n <= self.__num_hidden_layers + 1

      for i in range(0, n):
          w = self._w(i + 1)
          b = self._b(i + 1)
          if i < self.__num_hidden_layers:
              last_output = self._activate(last_output, w, b)
          else:
              last_output = self._activate(last_output, w, b, is_target= True)

      return last_output




In [78]:
sess = tf.Session()
with sess.graph.as_default():
    NUM_GENES_1 = 20000
    NUM_HIDDEN = 2
    NUM_CLASSES = 6
    NUM_NODES = [1024, 512]

    ae = AutoEncoder([NUM_GENES_1] + NUM_NODES + [NUM_CLASSES])
    ae._setup_variables(initialize = True, initialize_data = results[seed][0])

In [79]:
import scipy.io

# Load mat file with desired data
mat = scipy.io.loadmat('BRCA1View20000.mat')

# Access data from the mat file


In [80]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Mon Oct 28 10:46:10 2019',
 '__version__': '1.0',
 '__globals__': [],
 'GII': array([[   0.        , 2750.21122462, 1115.55242729, ..., 2851.57913616,
         1894.77504769, 6245.72295654]]),
 'data': array([[ 6.36537394, 14.33044161, 14.38833129, ...,  9.2944014 ,
         14.46428166,  6.73075214],
        [ 5.61056136,  7.97913885,  6.01671468, ..., 13.70304895,
         13.20370814,  6.87978212],
        [ 5.13479894, 10.24424041, 12.16443468, ...,  6.06361747,
         14.16980399,  6.13441445],
        ...,
        [ 5.24246002,  5.39674418,  5.69402156, ...,  5.23942778,
          5.23571632,  5.12363131],
        [ 5.19776742,  5.35215797,  5.45974413, ...,  5.39540163,
          5.15554335,  5.2103074 ],
        [ 5.64353704,  5.81181326,  5.26901508, ...,  5.5012646 ,
          5.4426976 ,  5.45647209]]),
 'gene': array([[array(['SCGB2A2'], dtype='<U7'), array(['MUCL1'], dtype='<U5'),
         array(['SCGB1D

In [101]:
df = pd.DataFrame()

In [82]:
len(mat['data']) #20,000
len(mat['data'][0]) #2133
len(mat['gene'][0]) #1
cols = []

for g in mat['gene'][0]:
    cols.append(g[0])

In [116]:
len(mat['gene'][0])

20000

In [99]:
len(cols)

20000

In [117]:
len(mat['data'])

20000

In [103]:
i = 0
for c in cols:
    df2 = pd.DataFrame(mat['data'][i],columns = [c])
    df = pd.concat([df, df2], axis=1)
    i += 1;
    if i%1000 ==0:
        print(str((i/20000)*100) + '% Transfered')


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000


In [105]:
labels = []
for l in mat['targets']:
    labels.append(l[0])

df['label'] = labels

/tmp/ipykernel_22385/2702621230.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['label'] = labels


In [111]:
len(labels)

2133

In [112]:
len(df)

2133

In [113]:
len(df.columns)

21445

In [110]:
df.columns.nunique()

20001

In [114]:
df = df.loc[:, ~df.columns.duplicated(keep='first')]


In [115]:
df

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,...,LOC641311,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294,label
0,6.365374,5.610561,5.134799,5.735802,5.855315,5.443985,8.005882,5.615179,6.682096,5.756946,...,5.409726,5.420998,5.396492,5.358751,5.424770,5.411055,5.242460,5.197767,5.643537,6
1,14.330442,7.979139,10.244240,7.138279,9.890661,11.195867,13.511164,10.548596,5.491578,6.805606,...,5.213895,5.167652,5.273045,5.287980,5.398061,5.226934,5.396744,5.352158,5.811813,4
2,14.388331,6.016715,12.164435,5.272691,6.468386,5.265518,13.386034,8.764722,7.331663,11.967719,...,5.707537,5.327874,5.549589,5.333051,5.368544,5.296182,5.694022,5.459744,5.269015,4
3,13.111243,7.251862,8.389121,8.664654,10.378585,5.544646,11.159766,8.263224,5.497906,5.923506,...,5.351022,5.009093,5.345468,5.176322,5.326991,5.480259,5.292691,5.242160,5.635111,5
4,12.243103,7.293489,10.919581,9.566137,8.503568,11.492583,11.531352,9.818938,9.443134,6.889670,...,5.320883,5.487783,5.479967,5.332074,5.273141,5.282872,5.222482,5.186136,5.616094,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,7.199176,5.708891,5.672914,14.464282,9.816897,5.604835,13.303832,6.391817,13.010996,8.294409,...,5.695733,5.363557,5.394837,5.442947,5.423763,5.558225,5.575000,5.636068,5.454823,3
2129,10.354497,14.464282,7.633244,14.464282,11.294018,5.468214,10.886420,12.260033,5.662372,6.274380,...,5.537254,5.613097,5.254552,5.205904,5.486546,5.206480,5.161718,5.451296,5.215587,5
2130,9.294401,13.703049,6.063617,10.063176,5.393440,5.529584,10.937517,12.220400,9.985646,8.370227,...,5.520596,5.572326,5.317225,5.178158,5.316757,5.210332,5.239428,5.395402,5.501265,2
2131,14.464282,13.203708,14.169804,13.731721,12.478907,5.347654,14.464282,9.593148,6.165212,11.207149,...,5.650387,5.359733,5.374522,5.245155,5.453742,5.353135,5.235716,5.155543,5.442698,5


In [118]:
mat['id'][0]

array([2080,  236,  741, ...,  698, 1626,  694], dtype=uint16)

In [121]:
df = df.set_index(pd.Index(mat['id'][0]))
df.head()

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,...,LOC641311,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294,label
2080,6.365374,5.610561,5.134799,5.735802,5.855315,5.443985,8.005882,5.615179,6.682096,5.756946,...,5.409726,5.420998,5.396492,5.358751,5.424770,5.411055,5.242460,5.197767,5.643537,6
236,14.330442,7.979139,10.244240,7.138279,9.890661,11.195867,13.511164,10.548596,5.491578,6.805606,...,5.213895,5.167652,5.273045,5.287980,5.398061,5.226934,5.396744,5.352158,5.811813,4
741,14.388331,6.016715,12.164435,5.272691,6.468386,5.265518,13.386034,8.764722,7.331663,11.967719,...,5.707537,5.327874,5.549589,5.333051,5.368544,5.296182,5.694022,5.459744,5.269015,4
380,13.111243,7.251862,8.389121,8.664654,10.378585,5.544646,11.159766,8.263224,5.497906,5.923506,...,5.351022,5.009093,5.345468,5.176322,5.326991,5.480259,5.292691,5.242160,5.635111,5
1446,12.243103,7.293489,10.919581,9.566137,8.503568,11.492583,11.531352,9.818938,9.443134,6.889670,...,5.320883,5.487783,5.479967,5.332074,5.273141,5.282872,5.222482,5.186136,5.616094,3


In [123]:
with sess.graph.as_default():
  data = tf.constant(df.values[:,:-1], dtype=tf.float32)
  logits_validation = ae.supervised_net(data, NUM_HIDDEN+1)
  sess.run(tf.global_variables_initializer())
  sess.run(logits_validation)

2023-04-30 22:34:56.052860: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [124]:
pred = tf.argmax(logits_validation, 1).eval(session=sess)+1
pred

array([1, 1, 1, ..., 1, 1, 1])

In [126]:
np.unique(pred)

array([1, 2, 6])

In [ ]:
# look at these preds against those in results, are they the same?

In [127]:
print('Accuracy:',sum(df['label'] == pred)/len(pred))

Accuracy: 0.19503047351148617


In [128]:
for key in results.keys():
  print("Accuracy for %s:"%key,results[key][1])

Accuracy for rs_01: 0.6741678


In [131]:
# gene_labels = pd.read_csv('/content/drive/My Drive/DeepType_results/gene_labels.txt',header=None)[0].tolist()
gene_labels = cols
pd.Series(cols)

0          SCGB2A2
1            MUCL1
2          SCGB1D2
3              PIP
4        LOC648852
           ...    
19995    LOC642453
19996    LOC652100
19997    LOC646050
19998    LOC644912
19999    LOC652294
Length: 20000, dtype: object

In [132]:
L2 = {}
for alpha in results.keys():
  L2[alpha] = pd.Series(np.sqrt(np.sum(results[alpha][0]['w_1']**2,axis=1)),index=gene_labels)
L2[alpha]

SCGB2A2      0.104677
MUCL1        0.065927
SCGB1D2      0.113932
PIP          0.109027
LOC648852    0.596426
               ...   
LOC642453    0.018213
LOC652100    0.020264
LOC646050    0.020783
LOC644912    0.018602
LOC652294    0.017639
Length: 20000, dtype: float32

In [142]:
L2[alpha].sort_values()

MYL2      0.002842
MYL1      0.002891
TAGLN3    0.002897
SPANXE    0.003047
LCE2B     0.003064
            ...   
GABRP     1.111665
ESR1      1.280077
KRT14     1.320521
SFRP1     1.374987
KRT17     1.433328
Length: 20000, dtype: float32

In [143]:
L2

{'rs_01': SCGB2A2      0.104677
 MUCL1        0.065927
 SCGB1D2      0.113932
 PIP          0.109027
 LOC648852    0.596426
                ...   
 LOC642453    0.018213
 LOC652100    0.020264
 LOC646050    0.020783
 LOC644912    0.018602
 LOC652294    0.017639
 Length: 20000, dtype: float32}

In [133]:
def apply_thres(l2,thres = 0.1):
  l2 = l2.loc[l2 > thres]
  return l2

In [134]:
L2_thres = {}
for alpha in results.keys():
  L2_thres[alpha] = apply_thres(L2[alpha])

In [135]:
L2_thres[alpha]

SCGB2A2      0.104677
SCGB1D2      0.113932
PIP          0.109027
LOC648852    0.596426
TFF3         0.503756
               ...   
FAM62C       0.103613
ACTR3B       0.113537
LOC653110    0.114811
GPR125       0.118789
GPR109B      0.100625
Length: 517, dtype: float32

In [136]:
sets = {}
for alpha in results.keys():
  sets[alpha] = set(L2_thres[alpha].index)
  print("Number of genes alpha=%s:"%alpha,len(sets[alpha]))

Number of genes alpha=rs_01: 517


In [138]:
!pip install matplotlib-venn

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for matplotlib-venn: filename=matplotlib_venn-0.11.9-py3-none-any.whl size=32999 sha256=e400aea5a9266027ef40945200802e749ec628d1ae0710f09aed199c8d318e82
  Stored in directory: /home/jupyter-ajakusov/.cache/pip/wheels/25/de/d4/29dfc5d4520b956df7bc54a8464ad053042918bc525e88bf66
Successfully built matplotlib-venn

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [139]:
from matplotlib_venn import venn3

venn3(sets.values(),set_labels=sets.keys());

TypeError: bad operand type for abs(): 'dict_values'